In [39]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.neighbors import NearestNeighbors

df_profile = pd.read_csv('C:\\Users\\SUVARNA\\Food_recommendation\\Recommendation\\user_Profiles.csv')
df_activity = pd.read_csv('C:\\Users\\SUVARNA\\Food_recommendation\\DJANGO\\minor\\website\\csvfile\\user_activity.csv')
df_profile.head(3)

,User_Id,Veg_Non,Nutrient,Disease,Diet
0,User_1,non-veg,chloride,anemia kidney_disease goitre rickets pregnancy,type_a_diet high_protien_diet low_carb_diet l...
1,User_2,veg,chloride,goitre,high_fiber_diet vegan_diet
2,User_3,veg,magnesium,cancer hypertension goitre heart_disease scurvy,high_fiber_diet ketogenic_diet high_protien_diet


In [40]:
df_activity.head(3)

,User_Id,Meal_Id,Name,catagory,description,Veg_Non,Review,Nutrient,Disease,Diet,Price
0,User_49,meal_id1,summer squash salad,salad,"white balsamic vinegar, lemon juice, lemon rin...",veg,5,fiber,obesity diabeties goitre hypertension,high_protien_diet high_fiber_diet ketogenic_d...,495
1,User_39,meal_id2,chicken minced salad,salad,"olive oil, chicken mince, garlic (minced), oni...",non-veg,10,fiber,kidney_disease obesity hypertension anemia go...,type_a_diet high_protien_diet high_fiber_diet...,435
2,User_77,meal_id3,sweet chilli almonds,chilli,"almonds whole, egg white, curry leaves, salt, ...",veg,1,vitamin_a,kidney_disease hypertension goitre heart_dise...,high_protien_diet high_fiber_diet paleo_diet ...,390


In [41]:
class Recommender:
    
    def __init__(self):
        self.df = pd.read_csv('C:\\Users\\SUVARNA\\Food_recommendation\\Recommendation\\user_Profiles.csv')
        self.activity = pd.read_csv('C:\\Users\\SUVARNA\\Food_recommendation\\DJANGO\\minor\\website\\csvfile\\user_activity.csv')
    
    def get_features(self):
        #getting dummies of dataset
        nutrient_dummies = self.df.Nutrient.str.get_dummies()
        disease_dummies = self.df.Disease.str.get_dummies(sep=' ')
        diet_dummies = self.df.Diet.str.get_dummies(sep=' ')
        feature_df = pd.concat([nutrient_dummies,disease_dummies,diet_dummies],axis=1)
     
        return feature_df
    
    def k_neighbor(self,inputs):
        
        feature_df = self.get_features()
        
        #initializing model with k=20 neighbors
        model = NearestNeighbors(n_neighbors=6,algorithm='ball_tree')
        
        # fitting model with dataset features
        model.fit(feature_df)
        
        df_results = pd.DataFrame(columns=list(self.df.columns))
        
      
        # getting distance and indices for k nearest neighbor
        distnaces , indices = model.kneighbors(inputs)

        for i in list(indices):
            df_results = df_results.append(self.df.loc[i])
                
#         df_results = df_result
        
        df_results = df_results.reset_index(drop=True)
        return df_results
    
    def user_based(self,user_profile,user_id):
        
        data = self.get_features()
        total_features = data.columns
        d = dict()
        for i in total_features:
            d[i]= 0
        for i in user_profile:
            d[i] = 1
        final_input = list(d.values())
        
        similar_users = self.k_neighbor([final_input])
        users = list(similar_users.User_Id)
        results = self.activity[self.activity.User_Id.isin(users)] #taking acitivies
        results = results[results['User_Id']!=user_id] # selecting those which are not reviewed by user
        results = results.filter(['Name','Nutrient','Veg_Non','Price','Review','Diet','Disease','description'])

        results = results.drop_duplicates(subset=['Name'])
        results = results.reset_index(drop=True)
        return results

In [42]:
profile_A = ['calcium','iron','diabeties','gluten_free_diet']
user_id = 'User_71' # asumming for example

In [43]:
ob = Recommender()
data = ob.get_features()
total_features = data.columns
d = dict()
for i in total_features:
    d[i]= 0
for i in profile_A:
    d[i] = 1

final_input = list(d.values())
similar_users = ob.k_neighbor([final_input]) # pass 2d array []
similar_users

c:\Users\SUVARNA\AppData\Local\Programs\Python\Python3.9\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,User_Id,Veg_Non,Nutrient,Disease,Diet
0,User_23,veg,protien,hypertension,gluten_free_diet
1,User_9,veg,iron,hypertension,high_fiber_diet gluten_free_diet vegan_diet l...
2,User_26,veg,iron,hypertension,high_fiber_diet gluten_free_diet vegan_diet l...
3,User_32,veg,calcium,hypertension,high_protien_diet ketogenic_diet gluten_free_...
4,User_53,non-veg,iron,goitre,high_fiber_diet vegan_diet low_fat_diet
5,User_78,veg,calcium,hypertension,high_protien_diet ketogenic_diet gluten_free_...


In [44]:
recommendations = ob.user_based(profile_A,user_id) 
recommendations

c:\Users\SUVARNA\AppData\Local\Programs\Python\Python3.9\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,Name,Nutrient,Veg_Non,Price,Review,Diet,Disease,description
0,almond and cranberry poha,iron,veg,235,9,high_fiber_diet gluten_free_diet vegan_diet l...,hypertension,"almond flakes, onion, poha, cranberries (froze..."
1,almond pearls,protien,veg,200,6,gluten_free_diet,hypertension,"toasted almonds, blueberries, oats, corn flake..."
2,almond and amaranth ladoo,calcium,veg,520,4,high_protien_diet ketogenic_diet gluten_free_...,hypertension,"popped amaranth seeds, jaggery, almonds (slive..."
3,sweet potato and quinoa bowl,protien,veg,460,7,high_protien_diet high_fiber_diet gluten_free...,kidney_disease hypertension goitre scurvy pre...,"स्वीट पौटेटो क्यूब्स, नारियल, प्याज, क्निोआ (प..."
4,caramelized sesame smoked almonds,calcium,veg,255,5,high_protien_diet ketogenic_diet low_fat_diet...,heart_disease,"साबुत बादाम, सफेद और काले तिल, चीनी, रोजमेरी, ..."
5,carrot ginger soup,sodium,veg,340,1,high_fiber_diet alkaline_diet,obesity,"गाजर, जैतून का तेल, नमक, वेजिटेबल स्टॉक, अदरक ..."
6,riceless chicken biryani,vitamin_c,non-veg,415,5,type_a_diet high_protien_diet low_carb_diet l...,anemia kidney_disease goitre rickets pregnancy,"malabar peppercorn, cinnamon stick, green card..."
7,chicken sukka,chloride,non-veg,385,10,type_a_diet high_protien_diet low_carb_diet l...,anemia kidney_disease goitre rickets pregnancy,"chicken (small pieces), onion (thin slices), c..."
8,chilli chicken,chloride,non-veg,380,6,type_a_diet high_protien_diet low_carb_diet l...,anemia kidney_disease goitre rickets pregnancy,"boneless chicken, salt, cornflour, black peppe..."
9,chicken minced salad,fiber,non-veg,410,3,type_a_diet high_protien_diet high_fiber_diet...,kidney_disease obesity hypertension anemia go...,"जैतून का तेल, चिकन कीमा, लहसुन , प्याज, नमक, क..."
